# Analysis of Nighttime Power Use

The nighttime period included in this analysis is Midnight - 6 am; power consumption is averaged across this period.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display, Markdown, Image, SVG
import bmondata
import scrapbook as sb

In [ ]:
# These parameters are changed by the report creation script.  All buildings
# and servers are cycled through.
building_id = 3
server_web_address = 'https://bmon.analysisnorth.com'

In [ ]:
# This report, at the moment, only applies to one building.  Raise an error,
# which will cause the report to abort, if we are processing a building other
# than the Mitchell home.
if building_id != 3 or ('analysisnorth' not in server_web_address):
    raise RuntimeError('Not Applicable to this Building')

In [ ]:
# The report generator script needs to the know the Title of this report and where
# to sort it with respect to other reports.  Glue that information to this notebook.
sb.glue('title', 'Nighttime Nanny')
sb.glue('sort_order', 10)

In [ ]:
# Make a Server object that allows us to pull data from the BMON site.
server = bmondata.Server(server_web_address)

In [ ]:
# Pull down hourly average electric use data from the BMON server.
electric_meter_id = 'cea_L131776533'     # sensor ID of the Mitchell electric meter
df = server.sensor_readings(electric_meter_id, averaging='1H')

# Change column name to "power", but also create a new column that contains the
# hour of the day (0 - 23) for each row.
df.columns = ['power']
df['hour'] = df.index.hour

# Narrow the dataset down to just nighttime hours
df_night = df.query('hour >=0 and hour <= 5')

# Average the nighttime hours for each day in the dataset
df_night_dayavg = df_night.resample('1D').mean()

# Now we can drop the hour column, but also eliminate outlier days with usage
# over 0.75 kW
df_night_dayfinal = df_night_dayavg.drop(columns=['hour']).query('power < 0.75')

# Plot all these daily averages, along with average value
daily_plot = go.Scatter(x=df_night_dayfinal.index, y=df_night_dayfinal.power, name='Nighttime Power')
avg_power = df_night_dayfinal.power.mean()
avg_plot = go.Scatter(
    x=[df_night_dayfinal.index[0], df_night_dayfinal.index[-1]], 
    y=[avg_power, avg_power],
    name='Nighttime Average',
    mode='lines'
)
fig = go.Figure(
    data=[daily_plot, avg_plot],
    layout_title_text = 'Daily Average Nighttime Power',
    layout_yaxis_title_text = 'Nighttime Power, kW',
)
fig.show()

In [ ]:
# Looks like there is seasonality to these values.  So, create a
# monthly profile (average nighttime use for each month, 1 - 12).
# Then, plot the average of the last 7 days as a dot superimposed 
# on that plot.
df_night_dayfinal['month'] = df_night_dayfinal.index.month
df_mo = df_night_dayfinal.groupby('month').mean()      # average for each month
plot_mo = go.Scatter(
    x=df_mo.index, 
    y=df_mo.power,
    name='Monthly Profile',
    mode='lines'
)
# last 7 days, not using very last day, as it is probably partial
ser_avg_last_week = df_night_dayfinal[-8:-1].mean()  
plot_last_week = go.Scatter(
    x=[ser_avg_last_week.month], 
    y=[ser_avg_last_week.power],
    name='Last Week',
    marker_size = 10
)
fig = go.Figure(
    data=[plot_mo, plot_last_week],
    layout_title_text = 'Last 7 Days vs. Typical Nighttime Power for this Month',
    layout_xaxis_title_text = 'Month',
    layout_yaxis_title_text = 'Nighttime Power, kW'
)
fig.show()